In [2]:
import numpy as np
import data.load

train_set, valid_set, dicts = data.load.atisfull()
w2idx, labels2idx = dicts['words2idx'], dicts['labels2idx']
#print(w2idx)
#print(labels2idx)

train_x, _, train_label = train_set
val_x, _, val_label = valid_set

# Create index to word/label dicts
idx2w  = {w2idx[k]:k for k in w2idx}
idx2la = {labels2idx[k]:k for k in labels2idx}

print("--------------------------------")
#print(idx2w)
#print(idx2la)


print(train_x[0:10])

# For conlleval script
words_train = [ list(map(lambda x: idx2w[x], w)) for w in train_x]
labels_train = [ list(map(lambda x: idx2la[x], y)) for y in train_label]
words_val = [ list(map(lambda x: idx2w[x], w)) for w in val_x]
labels_val = [ list(map(lambda x: idx2la[x], y)) for y in val_label]

n_classes = len(idx2la)
n_vocab = len(idx2w)

print("--------------------------")
print("-------------------------")


print(words_train[2])
print(labels_train[2])

print("------------------------")
print("-------------------------")
print("--------------------------")
print(words_val[800])
print(labels_val[2])


--------------------------------
[array([232, 542, 502, 196, 208,  77,  62,  10,  35,  40,  58, 234, 137,
        62,  11, 234, 481, 321]), array([554, 194,  50,  66, 208, 379, 502,  69, 358, 496, 321]), array([554, 241, 481,  56, 500, 234, 415, 205, 200, 481,  10,  35, 193,
       270, 543]), array([ 99,  24, 208, 466, 502, 367]), array([410, 515, 183, 208, 379, 502, 376, 529,  11, 158]), array([232, 331,  13, 193, 504, 208, 114, 502, 313]), array([554, 252, 353,  23, 241, 534, 358,  13, 193, 208, 107, 502, 128]), array([439, 301, 481, 194, 208, 379, 502, 285,  41, 358, 496]), array([ 32, 194, 208,  77, 502, 543]), array([554, 252, 353, 217, 512, 241,  66, 234, 137])]
--------------------------
-------------------------
['what', 'is', 'the', 'arrival', 'time', 'in', 'san', 'francisco', 'for', 'the', 'DIGITDIGITDIGIT', 'am', 'flight', 'leaving', 'washington']
['O', 'O', 'O', 'B-flight_time', 'I-flight_time', 'O', 'B-fromloc.city_name', 'I-fromloc.city_name', 'O', 'O', 'B-depart_time.ti

In [3]:
print("Example sentence : {}".format(words_train[0]))
print("Encoded form: {}".format(train_x[0]))
print()
print("It's label : {}".format(labels_train[0]))
print("Encoded form: {}".format(train_label[0]))
print(type(words_train))

Example sentence : ['i', 'want', 'to', 'fly', 'from', 'boston', 'at', 'DIGITDIGITDIGIT', 'am', 'and', 'arrive', 'in', 'denver', 'at', 'DIGITDIGITDIGITDIGIT', 'in', 'the', 'morning']
Encoded form: [232 542 502 196 208  77  62  10  35  40  58 234 137  62  11 234 481 321]

It's label : ['O', 'O', 'O', 'O', 'O', 'B-fromloc.city_name', 'O', 'B-depart_time.time', 'I-depart_time.time', 'O', 'O', 'O', 'B-toloc.city_name', 'O', 'B-arrive_time.time', 'O', 'O', 'B-arrive_time.period_of_day']
Encoded form: [126 126 126 126 126  48 126  35  99 126 126 126  78 126  14 126 126  12]
<class 'list'>


In [ ]:
from keras.models import Sequential
from keras.layers.embeddings import Embedding
from keras.layers.recurrent import SimpleRNN
from keras.layers.core import Dense, Dropout
from keras.layers.wrappers import TimeDistributed
from keras.layers import Convolution1D, GRU

model = Sequential()
model.add(Embedding(n_vocab,100))
model.add(Convolution1D(128, 5, border_mode='same', activation='relu'))
model.add(Dropout(0.25))
model.add(GRU(100,return_sequences=True))
model.add(TimeDistributed(Dense(n_classes, activation='softmax')))
model.compile('rmsprop', 'categorical_crossentropy')

C:\Users\jfver\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
C:\Users\jfver\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: UserWarning: Update your `Conv1D` call to the Keras 2 API: `Conv1D(128, 5, activation="relu", padding="same")`
  # Remove the CWD from sys.path while we load stuff.


In [ ]:
print(train_x[0:1])
print(n_classes)

import progressbar
n_epochs = 30

for i in range(n_epochs):
    print("Training epoch {}".format(i))
    
    bar = progressbar.ProgressBar(maxval=len(train_x))
    for n_batch, sent in bar(enumerate(train_x)):
        label = train_label[n_batch]
        # Make labels one hot
        label = np.eye(n_classes)[label][np.newaxis,:]
        # View each sentence as a batch
        sent = sent[np.newaxis,:]
        
        if sent.shape[1] > 1: #ignore 1 word sentences
            model.train_on_batch(sent, label)



N/A% (0 of 4978) |                       | Elapsed Time: 0:00:00 ETA:  --:--:--

[array([232, 542, 502, 196, 208,  77,  62,  10,  35,  40,  58, 234, 137,
        62,  11, 234, 481, 321])]
127
Training epoch 0


In [5]:

from metrics.accuracy import conlleval

labels_pred_val = []

bar = progressbar.ProgressBar(maxval=len(val_x))
for n_batch, sent in bar(enumerate(val_x)):
    label = val_label[n_batch]
    label = np.eye(n_classes)[label][np.newaxis,:]
    sent = sent[np.newaxis,:]

    pred = model.predict_on_batch(sent)
    pred = np.argmax(pred,-1)[0]
    labels_pred_val.append(pred)

labels_pred_val = [ list(map(lambda x: idx2la[x], y)) \
                                    for y in labels_pred_val]
# Predictions, Ground Truths, Corresponding Words, Name of the files where the predictions are made.
con_dict = conlleval(labels_pred_val, labels_val, 
                            words_val, 'measure.txt')

print('Precision = {}, Recall = {}, F1 = {}'.format(
            con_dict['r'], con_dict['p'], con_dict['f1']))


"\nfrom metrics.accuracy import conlleval\n\nlabels_pred_val = []\n\nbar = progressbar.ProgressBar(maxval=len(val_x))\nfor n_batch, sent in bar(enumerate(val_x)):\n    label = val_label[n_batch]\n    label = np.eye(n_classes)[label][np.newaxis,:]\n    sent = sent[np.newaxis,:]\n\n    pred = model.predict_on_batch(sent)\n    pred = np.argmax(pred,-1)[0]\n    labels_pred_val.append(pred)\n\nlabels_pred_val = [ list(map(lambda x: idx2la[x], y))                                     for y in labels_pred_val]\n# Predictions, Ground Truths, Corresponding Words, Name of the files where the predictions are made.\ncon_dict = conlleval(labels_pred_val, labels_val, \n                            words_val, 'measure.txt')\n\nprint('Precision = {}, Recall = {}, F1 = {}'.format(\n            con_dict['r'], con_dict['p'], con_dict['f1']))\n"

Code for loading AWS Data